## Exercise: Deploy Model Solution

In [1]:
import boto3
import json
import sagemaker
import zipfile

# Function below unzips the archive to the local directory. 

def unzip_data(input_data_path):
    with zipfile.ZipFile(input_data_path, 'r') as input_data_zip:
        input_data_zip.extractall('.')

# Input data is a file with a single JSON object per line with the following format: 
# {
#  "reviewerID": <string>,
#  "asin": <string>,
#  "reviewerName" <string>,
#  "helpful": [
#    <int>, (indicating number of "helpful votes")
#    <int>  (indicating total number of votes)
#  ],
#  "reviewText": "<string>",
#  "overall": <int>,
#  "summary": "<string>",
#  "unixReviewTime": <int>,
#  "reviewTime": "<string>"
# }
# 
# We are specifically interested in the fields "helpful" and "reviewText"
#

def label_data(input_data):
    labeled_data = []
    HELPFUL_LABEL = "__label__1"
    UNHELPFUL_LABEL = "__label__2"
     
    for l in open(input_data, 'r'):
        l_object = json.loads(l)
        helpful_votes = float(l_object['helpful'][0])
        total_votes = l_object['helpful'][1]
        reviewText = l_object['reviewText']
        if total_votes != 0:
            if helpful_votes / total_votes > .5:
                labeled_data.append(" ".join([HELPFUL_LABEL, reviewText]))
            elif helpful_votes / total_votes < .5:
                labeled_data.append(" ".join([UNHELPFUL_LABEL, reviewText]))
          
    return labeled_data


# Labeled data is a list of sentences, starting with the label defined in label_data. 

def split_sentences(labeled_data):
    new_split_sentences = []
    for d in labeled_data:       
        sentences = " ".join(d.split()[1:]).split(".") # Initially split to separate label, then separate sentences
        for s in sentences:
            if s: # Make sure sentences isn't empty. Common w/ "..."
                new_split_sentences.append(s)
    return new_split_sentences


unzip_data('reviews_Musical_Instruments_5.json.zip')
labeled_data = label_data('reviews_Musical_Instruments_5.json')
new_split_sentence_data = split_sentences(labeled_data)

print(new_split_sentence_data[0:9])

['The product does exactly as it should and is quite affordable', 'I did not realized it was double screened until it arrived, so it was even better than I had expected', "As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording", ' :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]', 'The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeable reduction of volume or high frequencies', ' The double cloth filter blocks the pops and lets the voice through with no coloration', ' The metal clamp mount attaches to the mike stand secure enough to keep it attached', ' The goose neck needs a little coaxing to stay where you put

In [3]:
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker import image_uris

role = get_execution_role()

image_uri = image_uris.retrieve(framework='blazingtext',region='us-east-1')

model_data = "s3://udacity-sagemaker-solutiondata20212021/l2e1/model_artifact/blazingtext-trainingjob-1/output/model.tar.gz"

model = Model(image_uri=image_uri, model_data=model_data, role=role)

predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.large")

-----!

## Exercise: Evaluate Data Solution

In [5]:
from sagemaker.predictor import Predictor
import json

predictor = Predictor("blazingtext-2022-09-14-11-44-17-897")

example_sentences = new_split_sentence_data[0:5]

payload = {"instances": example_sentences}

print(json.dumps(payload))

predictions = json.loads(predictor.predict(json.dumps(payload), initial_args={'ContentType': 'application/json'}))

{"instances": ["The product does exactly as it should and is quite affordable", "I did not realized it was double screened until it arrived, so it was even better than I had expected", "As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording", " :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]", "The primary job of this device is to block the breath that would otherwise produce a popping sound, while allowing your voice to pass through with no noticeable reduction of volume or high frequencies"]}


In [6]:
predictions

[{'label': ['__label__1'], 'prob': [0.8796815276145935]},
 {'label': ['__label__1'], 'prob': [0.8387504816055298]},
 {'label': ['__label__1'], 'prob': [0.8774088025093079]},
 {'label': ['__label__1'], 'prob': [0.8051218390464783]},
 {'label': ['__label__1'], 'prob': [0.9146125316619873]}]

In [7]:
predictor.delete_endpoint()